In [0]:
from pyspark.sql.functions import md5, concat_ws, col, explode, split

In [0]:
wifi_locations_df = spark.read \
    .table("mta_bronze.wifi_location")

dim_lines_df = spark.read \
    .table("mta_silver.dim_line")

In [0]:
wifi_line_df = wifi_locations_df.select(
    'wfi_station',
    'wfi_station_complex',
    'wfi_lines') \
    .withColumn("wfi_sk", md5(concat_ws("_", col("wfi_station"), col("wfi_station_complex"), col("wfi_lines")))) \
    .withColumn("wfi_lines", explode(split(wifi_locations_df["wfi_lines"], ""))) \
    .select(
        "wfi_sk",
        "wfi_lines"
    )

In [0]:
wifi_line_final_df = wifi_line_df \
    .join(dim_lines_df, wifi_line_df.wfi_lines == dim_lines_df.lin_nk, "left") \
    .select(
        "wfi_sk",
        "lin_sk"
    )

In [0]:
wifi_line_final_df.write.mode("overwrite").format("delta").option("mergeSchema", "true").saveAsTable("mta_silver.lkp_wifi_line")

In [0]:
%sql
SELECT * FROM mta_silver.lkp_wifi_line;

In [0]:
dbutils.notebook.exit("Success")